**IMPORT SECTION**

In [215]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
!pip install beautifulsoup4
!pip install wikipedia

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [216]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import string
import math
import matplotlib.pyplot as plt
import reimport nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import string
import math
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import re
import scipy
import math
import wikipedia
import requests
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.manifold import MDS
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.fasttext import FastText
import scipy
import math
import requests
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.manifold import MDS
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

**GET TEXT FROM WIKIPEDIA**

In [217]:
titles=wikipedia.search('spiderman')
result=wikipedia.page(titles)

In [ ]:
print("len of docs",len(result.content))
print("type of docs",type(result.content))
print("content",result.content)

In [ ]:
def charbased_preprocessing(content):
    tokens = []
    for char in content:
        tokens.append(char)
    return(tokens)
    

In [ ]:
chars=np.array(charbased_preprocessing(result.content)).reshape(-1,1)

In [ ]:
encoder = OneHotEncoder()
chars = encoder.fit_transform(chars).toarray()
print(chars)

In [ ]:
chars.shape

**DIVIDE TEXT TO SEQUENCES**

In [ ]:
x=[]
y=[]
sequence_length = 10
for i in range(0,chars.shape[0]-sequence_length):
    x.append(chars[i:i+sequence_length])
    y.append(chars[i+sequence_length])
x=np.array(x)
y=np.array(y)

In [ ]:
print(x.shape)
print(y.shape)

**INDEX TO CHAR DICT**

In [ ]:
dic = set([c for c in result.content])
dic=sorted(dic)
index2char = dict((i, c) for i, c in enumerate(dic))

In [ ]:
index2char

**TRAINING PROCESS**

In [ ]:
import tensorflow as tf

# Define the size of the input and output layers
input_size = chars.shape[1] 
output_size = chars.shape[1]
# Define the hyperparameters
hidden_size = 100  # Size of the hidden state
sequence_length = 10  # Length of the input sequence
learning_rate = 0.001

# Define the RNN model with multiple layers
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(hidden_size, return_sequences=True, input_shape=(sequence_length, input_size)),
    tf.keras.layers.SimpleRNN(hidden_size, return_sequences=True),
    tf.keras.layers.SimpleRNN(hidden_size),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

In [ ]:
history=model.fit(x, y, batch_size=64, epochs=50)

**PREDICT NEXT CHAR TO GENERATE TEXT**

In [ ]:
# Choose a random sequence from x
random_index = np.random.randint(0, len(x))
random_sequence = x[random_index]

# Reshape the random sequence to fit the model input shape
random_sequence = np.reshape(random_sequence, (1, sequence_length, input_size))

predicted_char_prob = model.predict(random_sequence)
predicted_char_index = np.argmax(predicted_char_prob)

# Convert the predicted character index to the actual character using index2char dictionary
predicted_char = index2char[predicted_char_index]

# Convert the random sequence back to characters using index2char dictionary
sequence_chars = [index2char[np.argmax(char)] for char in random_sequence[0]]

# Print the random sequence and the predicted character
print("Random Sequence:", ''.join(sequence_chars))
print("Predicted Next Character:", predicted_char)


**CHAR-BASED GENERATOR**

In [ ]:
sequence_index = np.random.randint(0, len(chars) - sequence_length)
sequence_chars = [index2char[np.argmax(char)] for char in chars[sequence_index:sequence_index + sequence_length]]

generated_text = ''.join(sequence_chars) 
random_index = np.random.randint(0, len(x))
random_sequence = x[random_index]

for _ in range(100):
    current_sequence = np.reshape(random_sequence, (1, sequence_length, input_size))
    predicted_char_prob = model.predict(current_sequence)
    predicted_char_index = np.argmax(predicted_char_prob)
    predicted_char = index2char[predicted_char_index]
    generated_text += predicted_char
    encoded_char=np.zeros(random_sequence.shape[1])
    encoded_char[predicted_char_index]=1
    random_sequence =np.vstack((random_sequence[1:],encoded_char))

print("Generated Text:", generated_text)
